# 학습 관련 기술

## 6.1 매개변수 갱신

### 6.1.1 확률적 경사 하강법(SGD)

$$ W \leftarrow W - \eta \frac{\partial L}{\partial W} $$

- W: 갱신할 가중치 매개변수
- $\frac{\partial L}{\partial W}$: W에 대한 손실 함수의 기울기
- $\eta$: 학습률
- $\leftarrow$: 우변의 값으로 좌변의 값을 갱신

In [1]:
# SGD 구현

class SGD:
  """Stochastic Gradient Descent
  
  Description:
    합습률을 인수로 받아 객체를 초기화한다.

  Methods:
    * __init__(lr=0.01): 학습률을 초기화한다.
    * update(params, grads): 매개변수와 기울기를 받아 매개변수를 갱신한다.
  """
  def __init__(self, lr=0.01):
    self.lr = lr
    
  def update(self, params, grads):
    for key in params.keys():
      params[key] -= self.lr * grads[key]


**SGD사용법**
```python
networks = TwoLayerNet(...)
optimizer = SGD(lr=0.01)

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) # 미니배치 획득
    loss = networks.loss(x_batch, t_batch)
    networks.backward()
    optimizer.update(networks.params, networks.grads)
    ...
```